In [ ]:
%pip install -U google-generativeai
%pip install google-ai-generativelanguage==0.6.15
%pip install -U langchain-google-genai
%pip install -U langchain-community
%pip install -U langgraph

In [ ]:
import os
import re
import google.generativeai as genai
from langgraph.graph import StateGraph, END
from typing import TypedDict

In [5]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [ ]:
client = genai.GenerativeModel("gemini-2.5-flash")

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        # Constrói o prompt com todo o histórico de mensagens
        prompt = ""
        for msg in self.messages:
            prompt += f"{msg['role']}: {msg['content']}\n"
        
        # Envia para o Gemini
        response = client.generate_content(prompt)
        return response.text
    
# Exemplo de uso
if __name__ == "__main__":
    agent = Agent(system="Você é um assistente útil e objetivo.")
    print(agent("Qual é a capital da França?"))

In [7]:
PROMPT_REACT = """
Você funciona em um ciclo de Pensamento, Ação, Pausa e Observação.
Ao final do ciclo, você fornece uma Resposta.
Use "Pensamento" para descrever seu raciocínio.
Use "Ação" para executar ferramentas - e então retorne "PAUSA".
A "Observação" será o resultado da ação executada.
Ações disponíveis:
    - consultar_estoque: retorna a quantidade disponível de um item no inventário (ex: "consultar_estoque: teclado")
    - consultar_preco_produto: retorna o preço unitário de um produto (ex: "consultar_preco_produto: mouse gamer")
    - encontrar_produto_mais_caro: retorna o nome e o preço do produto mais caro no inventario (não requer argumentos)

Exemplo:
Pergunta: Quantos monitores temos em estoque?
Pensamento: Devo consultar a ação consultar_estoque para saber a quantidade de monitores.
Ação: consultar_estoque: monitor
PAUSA

Observação: Temos 75 monitores em estoque.
Resposta: Há 75 monitores em estoque.

Exemplo:
Pergunta: Qual produto é o mais caro?
Pensamento: Preciso utilizar a ação encontrar_produto_mais_caro para descobrir qual produto tem o maior preço.
Ação: encontrar_produto_mais_caro
PAUSA

Observação: O produto mais caro é o(a) monitor com o preço de R$ 999.90.
Resposta: O produto mais caro é o(a) monitor com o preço de R$ 999.90.
""".strip()

In [8]:
class AgentState(TypedDict):
    pergunta: str
    historico: list[str]
    acao_pendente: str
    resposta_final: str

In [9]:
def consultar_estoque(item: str) -> str:
    """
    Simula a consulta de estoque de um item no inventário.
    """
    item = item.lower()
    estoque = {
        "monitor": 75,
        "teclado": 120,
        "mouse gamer": 80,
        "webcam": 40,
        "headset": 60,
        "impressora": 15
    }

    if item in estoque:
        return f"Temos {estoque[item]} {item}s em estoque." #criamos uma máscara para a resposta, para ficar mais elegante e profissional
    else:
        return f"Item '{item}' não encontrado no inventário." #criamos também uma resposta para produtos não localizados no inventário
    
def consultar_preco_produto(produto: str) -> str:
    """
    Simula a consulta do preço unitário de um produto.
    """
    produto = produto.lower()
    precos = {
        "monitor": 999.90,
        "teclado": 150.00,
        "mouse gamer": 99.50,
        "webcam": 120.00,
        "headset": 180.00,
        "impressora": 750.00
    }

    if produto in precos:
        return f"O preço de um(a) {produto} é R$ {precos[produto]:.2f}."
    else:
        return f"Produto '{produto}' não encontrado na lista de preços."

In [10]:
print(consultar_estoque("teclado"))

Temos 120 teclados em estoque.


In [11]:
print(consultar_preco_produto("impressora"))

O preço de um(a) impressora é R$ 750.00.


In [12]:
def ferramenta_encontrar_produto_mais_caro() -> str:
    """
    Retorna o nome e o preço do produto mais caro no inventário.
    Esta função não precisa de argumentos.
    """
    # dicionário de preços do inventário:
    precos_do_inventario = {
        "monitor": 999.90,
        "teclado": 150.00,
        "mouse gamer": 99.50,
        "webcam": 120.00,
        "headset": 180.00,
        "impressora": 750.00
    }

    if not precos_do_inventario:  # verificação de segurança
        return "Nenhum produto encontrado na lista de preços para comparação."

    # busca do produto mais caro
    nome_produto_mais_caro = max(precos_do_inventario, key=precos_do_inventario.get)
    valor_produto_mais_caro = precos_do_inventario[nome_produto_mais_caro]

    return f"O produto mais caro é o(a) {nome_produto_mais_caro} com preço de R$ {valor_produto_mais_caro:.2f}."

In [13]:
def run_react_agent(pergunta: str, max_iterations: int = 5) -> str:
    """
    Executa o ciclo ReAct para uma dada pergunta usando o modelo Gemini.
    """
    
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    chat = model.start_chat(history=[])
    chat.send_message(PROMPT_REACT)
    
    current_prompt = pergunta

    for i in range(max_iterations):
        response = chat.send_message(current_prompt)
        response_text = response.text.strip()
        
        print(f"\n--- Iteração {i+1} ---")
        print(f"Modelo pensou/respondeu:\n{response_text}\n")

        # Verifica se a resposta final está presente, mesmo que haja Pensamento antes.
        response_match_final = re.search(r"Resposta:\s*(.*)", response_text, re.DOTALL)
        if response_match_final:
            return response_match_final.group(1).strip()
        
        # Ajuste para lidar com ações com ou sem argumento (e sem o ':' para ações sem argumento)
        # # Ex: "Ação: nome_da_acao: argumento" OU "Ação: nome_da_acao"
        match = re.search(r"Ação:\s*(\w+)(?::\s*(.*(?:\n.*)*))?", response_text)

        if match:
            action_name = match.group(1).strip()
            action_arg = match.group(2).strip() if match.group(2) is not None else ""
            observacao_da_acao = ""

            if action_name == "consultar_estoque":
                observacao_da_acao = consultar_estoque(action_arg)
            elif action_name == "consultar_preco_produto":
                observacao_da_acao = consultar_preco_produto(action_arg)
            elif action_name == "encontrar_produto_mais_caro": #inclusão da nova ferramenta de comparação
                observacao_da_acao = ferramenta_encontrar_produto_mais_caro()
            else:
                observacao_da_acao = f"Erro: Ação '{action_name}' desconhecida. Verifique o prompt ou a implementação da ferramenta."
            
            # Simplificamos o prompt de Observação
            current_prompt = f"Observação: {observacao_da_acao}"
            
            print(f"Executou ação: {action_name} com argumento '{action_arg}'")
            print(f"Observação: {observacao_da_acao}\n")

        else:
            # Se o modelo não gerou uma Ação ou Resposta final clara, retorna um erro
            return f"Erro: O agente não conseguiu extrair uma Ação ou Resposta final após {i+1} iterações. Última resposta do modelo: {response_text}"
    
    # Se o limite de iterações for atingido sem uma resposta final
    return "Erro: Limite máximo de iterações atingido sem uma resposta final do agente."

In [14]:


def ferramenta_encontrar_produto_mais_caro() -> str:
    """
    Retorna o nome e o preço do produto mais caro no inventário.
    Esta função não precisa de argumentos.
    """
    # dicionário de preços do inventário:
    precos_do_inventario = {
        "monitor": 999.90,
        "teclado": 150.00,
        "mouse gamer": 99.50,
        "webcam": 120.00,
        "headset": 180.00,
        "impressora": 750.00
    }

    if not precos_do_inventario:  # verificação de segurança
        return "Nenhum produto encontrado na lista de preços para comparação."

    # busca do produto mais caro
    nome_produto_mais_caro = max(precos_do_inventario, key=precos_do_inventario.get)
    valor_produto_mais_caro = precos_do_inventario[nome_produto_mais_caro]

    return f"O produto mais caro é o(a) {nome_produto_mais_caro} com preço de R$ {valor_produto_mais_caro:.2f}."

In [15]:
# --- Exemplos de Interação com o Agente ---
print("--- Começando as Interações com o Agente ReAct ---")

--- Começando as Interações com o Agente ReAct ---


In [ ]:
# Interação 1: Consultar Estoque
pergunta_1 = "Quantos teclados temos em estoque?"
print(f"\n**Interação 1: {pergunta_1}**")
resposta_1 = run_react_agent(pergunta_1)
print(f"\n**RESPOSTA FINAL DO AGENTE 1:** {resposta_1}\n")
print("\n" + "="*80 + "\n")

In [ ]:
# Interação 2: Consultar Preço
pergunta_2 = "Qual o preço de um headset?"
print(f"\n**Interação 2: {pergunta_2}**")
resposta_2 = run_react_agent(pergunta_2)
print(f"\n**RESPOSTA FINAL DO AGENTE 2:** {resposta_2}\n")
print("\n" + "="*80 + "\n")

In [ ]:
# Interação 3: Item não encontrado no estoque
pergunta_3 = "Temos cadeiras em estoque?"
print(f"\n**Interação 3: {pergunta_3}**")
resposta_3 = run_react_agent(pergunta_3)
print(f"\n**RESPOSTA FINAL DO AGENTE 3:** {resposta_3}\n")
print("\n" + "="*80 + "\n")

In [ ]:
# Interação 4: Encontrar o Produto Mais Caro (A nova funcionalidade!)
pergunta_4 = "Qual é o produto mais caro?"
print(f"\n**Interação 4: {pergunta_4}**")
resposta_4 = run_react_agent(pergunta_4)
print(f"\n**RESPOSTA FINAL DO AGENTE 4:** {resposta_4}\n")
print("\n--- Fim das Interações ---")